In [1]:

# importing libraries
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
from bs4 import BeautifulSoup
import requests # library to handle requests
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# !conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
import geopy.geocoders # convert an address into latitude and longitude values

# !conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries are imported.')

Libraries are imported.


In [4]:
df_toronto = pd.read_csv('C:/Users/Only_Gentlemen/Downloads/toronto_base.csv')
df_toronto.head()

,Unnamed: 0,Post Code,Borough,Neighborhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [6]:

# for the city Toronto, latitude and longtitude are manually extracted via google search
toronto_latitude = 43.6932; toronto_longitude = -79.3832
map_toronto = folium.Map(location = [toronto_latitude, toronto_longitude], zoom_start = 10.7)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    

map_toronto

In [7]:
scarborough_data = df_toronto[df_toronto['Borough'] == 'Scarborough']
scarborough_data = scarborough_data.reset_index(drop=True).drop(columns = 'Unnamed: 0')
scarborough_data.head()

,Post Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [9]:
address_scar = 'Scarborough, Toronto'
latitude_scar = 43.773077
longitude_scar = -79.257774
print('The geograpical coordinate of "Scarborough" are: {}, {}.'.format(latitude_scar, longitude_scar))

map_Scarborough = folium.Map(location=[latitude_scar, longitude_scar], zoom_start=11.5)

# add markers to map
for lat, lng, label in zip(scarborough_data['Latitude'], scarborough_data['Longitude'], scarborough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius = 10,
        popup = label,
        color ='blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7).add_to(map_Scarborough)  
    
map_Scarborough

The geograpical coordinate of "Scarborough" are: 43.773077, -79.257774.


In [10]:
def foursquare_crawler (postal_code_list, neighborhood_list, lat_list, lng_list, LIMIT = 500, radius = 1000):
    result_ds = []
    counter = 0
    for postal_code, neighborhood, lat, lng in zip(postal_code_list, neighborhood_list, lat_list, lng_list):
         
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, 
            lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        tmp_dict = {}
        tmp_dict['Postal Code'] = postal_code; tmp_dict['Neighborhood(s)'] = neighborhood; 
        tmp_dict['Latitude'] = lat; tmp_dict['Longitude'] = lng;
        tmp_dict['Crawling_result'] = results;
        result_ds.append(tmp_dict)
        counter += 1
        print('{}.'.format(counter))
        print('Data is Obtained, for the Postal Code {} (and Neighborhoods {}) SUCCESSFULLY.'.format(postal_code, neighborhood))
    return result_ds;

In [11]:
CLIENT_ID = 'EA42K1JKMYY34TP1YYSVLC5DXQD0IOFRRY3MQGEAXZ44VVIT' # your Foursquare ID
CLIENT_SECRET = 'YGZ31K5ZFRLXOYA1B3ADUH5ZYLPNW2S3SBZ3ZMD05HCRQ2TM' # your Foursquare Secret
VERSION = '20180605'

In [14]:
print('Crawling different neighborhoods inside "Scarborough"')
Scarborough_foursquare_dataset = foursquare_crawler(list(scarborough_data['Post Code']),
                                                   list(scarborough_data['Neighborhood']),
                                                   list(scarborough_data['Latitude']),
                                                   list(scarborough_data['Longitude']),)

Crawling different neighborhoods inside "Scarborough"
1.
Data is Obtained, for the Postal Code M1B (and Neighborhoods Rouge, Malvern) SUCCESSFULLY.
2.
Data is Obtained, for the Postal Code M1C (and Neighborhoods Highland Creek, Rouge Hill, Port Union) SUCCESSFULLY.
3.
Data is Obtained, for the Postal Code M1E (and Neighborhoods Guildwood, Morningside, West Hill) SUCCESSFULLY.
4.
Data is Obtained, for the Postal Code M1G (and Neighborhoods Woburn) SUCCESSFULLY.
5.
Data is Obtained, for the Postal Code M1H (and Neighborhoods Cedarbrae) SUCCESSFULLY.
6.
Data is Obtained, for the Postal Code M1J (and Neighborhoods Scarborough Village) SUCCESSFULLY.
7.
Data is Obtained, for the Postal Code M1K (and Neighborhoods East Birchmount Park, Ionview, Kennedy Park) SUCCESSFULLY.
8.
Data is Obtained, for the Postal Code M1L (and Neighborhoods Clairlea, Golden Mile, Oakridge) SUCCESSFULLY.
9.
Data is Obtained, for the Postal Code M1M (and Neighborhoods Cliffcrest, Cliffside, Scarborough Village West) 

In [15]:
import pickle
with open("Scarborough_foursquare_dataset.txt", "wb") as fp:   #Pickling
    pickle.dump(Scarborough_foursquare_dataset, fp)
print('Received Data from Internet is Saved to Computer.')

Received Data from Internet is Saved to Computer.


In [16]:
with open("Scarborough_foursquare_dataset.txt", "rb") as fp:   # Unpickling
    Scarborough_foursquare_dataset = pickle.load(fp)

In [17]:
def get_venue_dataset(foursquare_dataset):
    result_df = pd.DataFrame(columns = ['Postal Code', 'Neighborhood', 
                                           'Neighborhood Latitude', 'Neighborhood Longitude',
                                          'Venue', 'Venue Summary', 'Venue Category', 'Distance'])
    # print(result_df)
    
    for neigh_dict in foursquare_dataset:
        postal_code = neigh_dict['Postal Code']; neigh = neigh_dict['Neighborhood(s)']
        lat = neigh_dict['Latitude']; lng = neigh_dict['Longitude']
        print('Number of Venuse in Coordination "{}" Posal Code and "{}" Negihborhood(s) is:'.format(postal_code, neigh))
        print(len(neigh_dict['Crawling_result']))
        
        for venue_dict in neigh_dict['Crawling_result']:
            summary = venue_dict['reasons']['items'][0]['summary']
            name = venue_dict['venue']['name']
            dist = venue_dict['venue']['location']['distance']
            cat =  venue_dict['venue']['categories'][0]['name']
            
            
            # print({'Postal Code': postal_code, 'Neighborhood': neigh, 
            #                   'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
            #                   'Venue': name, 'Venue Summary': summary, 
            #                   'Venue Category': cat, 'Distance': dist})
            
            result_df = result_df.append({'Postal Code': postal_code, 'Neighborhood': neigh, 
                              'Neighborhood Latitude': lat, 'Neighborhood Longitude':lng,
                              'Venue': name, 'Venue Summary': summary, 
                              'Venue Category': cat, 'Distance': dist}, ignore_index = True)
            # print(result_df)
    
    return(result_df)

In [18]:
scarborough_venues = get_venue_dataset(Scarborough_foursquare_dataset)

Number of Venuse in Coordination "M1B" Posal Code and "Rouge, Malvern" Negihborhood(s) is:
18
Number of Venuse in Coordination "M1C" Posal Code and "Highland Creek, Rouge Hill, Port Union" Negihborhood(s) is:
5
Number of Venuse in Coordination "M1E" Posal Code and "Guildwood, Morningside, West Hill" Negihborhood(s) is:
24
Number of Venuse in Coordination "M1G" Posal Code and "Woburn" Negihborhood(s) is:
9
Number of Venuse in Coordination "M1H" Posal Code and "Cedarbrae" Negihborhood(s) is:
27
Number of Venuse in Coordination "M1J" Posal Code and "Scarborough Village" Negihborhood(s) is:
12
Number of Venuse in Coordination "M1K" Posal Code and "East Birchmount Park, Ionview, Kennedy Park" Negihborhood(s) is:
27
Number of Venuse in Coordination "M1L" Posal Code and "Clairlea, Golden Mile, Oakridge" Negihborhood(s) is:
28
Number of Venuse in Coordination "M1M" Posal Code and "Cliffcrest, Cliffside, Scarborough Village West" Negihborhood(s) is:
11
Number of Venuse in Coordination "M1N" Pos

In [19]:
scarborough_venues.head()

,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
0,M1B,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,This spot is popular,Spa,595
1,M1B,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,This spot is popular,Paper / Office Supplies Store,735
2,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,600
3,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,Fast Food Restaurant,387
4,M1B,"Rouge, Malvern",43.806686,-79.194353,Harvey's,This spot is popular,Fast Food Restaurant,796


In [20]:
scarborough_venues.to_csv('C:/Users/Only_Gentlemen/Downloads/scarborough_venues.csv')

In [22]:
scarborough_venues = pd.read_csv('C:/Users/Only_Gentlemen/Downloads/scarborough_venues.csv')

In [23]:
neigh_list = list(scarborough_venues['Neighborhood'].unique())
print('Number of Neighborhoods inside Scarborough:')
print(len(neigh_list))
print('List of Neighborhoods inside Scarborough:')
neigh_list

Number of Neighborhoods inside Scarborough:
16
List of Neighborhoods inside Scarborough:


['Rouge, Malvern',
 'Highland Creek, Rouge Hill, Port Union',
 'Guildwood, Morningside, West Hill',
 'Woburn',
 'Cedarbrae',
 'Scarborough Village',
 'East Birchmount Park, Ionview, Kennedy Park',
 'Clairlea, Golden Mile, Oakridge',
 'Cliffcrest, Cliffside, Scarborough Village West',
 'Birch Cliff, Cliffside West',
 'Dorset Park, Scarborough Town Centre, Wexford Heights',
 'Maryvale, Wexford',
 'Agincourt',
 "Clarks Corners, Sullivan, Tam O'Shanter",
 "Agincourt North, L'Amoreaux East, Milliken, Steeles East",
 "L'Amoreaux West, Steeles West"]

In [24]:
neigh_venue_summary = scarborough_venues.groupby('Neighborhood').count()
neigh_venue_summary.drop(columns = ['Unnamed: 0']).head()

,Postal Code,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Venue Category,Distance
Neighborhood,,,,,,,
Agincourt,50,50,50,50,50,50,50
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",26,26,26,26,26,26,26
"Birch Cliff, Cliffside West",13,13,13,13,13,13,13
Cedarbrae,27,27,27,27,27,27,27
"Clairlea, Golden Mile, Oakridge",28,28,28,28,28,28,28


In [25]:

print('There are {} uniques categories.'.format(len(scarborough_venues['Venue Category'].unique())))

print('Here is the list of different categories:')
list(scarborough_venues['Venue Category'].unique())

There are 111 uniques categories.
Here is the list of different categories:


['Spa',
 'Paper / Office Supplies Store',
 'Fast Food Restaurant',
 'Caribbean Restaurant',
 'Coffee Shop',
 'Hobby Shop',
 'Filipino Restaurant',
 'Martial Arts Dojo',
 'African Restaurant',
 'Trail',
 'Chinese Restaurant',
 'Greek Restaurant',
 'Fruit & Vegetable Store',
 'Arts & Crafts Store',
 'Sandwich Place',
 'Park',
 'Burger Joint',
 'Italian Restaurant',
 'Breakfast Spot',
 'Playground',
 'Fried Chicken Joint',
 'Liquor Store',
 'Food & Drink Shop',
 'Pizza Place',
 'Smoothie Shop',
 'Beer Store',
 'Sports Bar',
 'Pharmacy',
 'Bank',
 'Discount Store',
 'Supermarket',
 'Grocery Store',
 'Indian Restaurant',
 'Electronics Store',
 'Hakka Restaurant',
 'Music Store',
 'Thai Restaurant',
 'Athletics & Sports',
 'Bakery',
 'Wings Joint',
 'Yoga Studio',
 'Asian Restaurant',
 'Bus Line',
 'Sporting Goods Shop',
 'Convenience Store',
 'Restaurant',
 'Train Station',
 'Japanese Restaurant',
 'Bowling Alley',
 'Department Store',
 'Hockey Arena',
 'Metro Station',
 'Light Rail Station

In [26]:
print(type(scarborough_venues[['Venue Category']]))

print(type(scarborough_venues['Venue Category']))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.series.Series'>


In [27]:
scarborough_onehot = pd.get_dummies(data = scarborough_venues, drop_first  = False, 
                              prefix = "", prefix_sep = "", columns = ['Venue Category'])
scarborough_onehot.head()

,Unnamed: 0,Postal Code,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Summary,Distance,African Restaurant,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,Badminton Court,Bakery,Bank,Bar,Beach,Beer Store,Bowling Alley,Breakfast Spot,Bubble Tea Shop,Burger Joint,Bus Line,Bus Station,Bus Stop,Café,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Electronics Store,Event Space,Fast Food Restaurant,Filipino Restaurant,Fish Market,Flea Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,General Entertainment,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym Pool,Hakka Restaurant,Hobby Shop,Hockey Arena,Hotpot Restaurant,Indian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motorcycle Shop,Music Store,Noodle House,Other Great Outdoors,Paper / Office Supplies Store,Park,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Rental Car Location,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Shop & Service,Shopping Mall,Skating Rink,Smoothie Shop,Soccer Field,Spa,Sporting Goods Shop,Sports Bar,Sri Lankan Restaurant,Supermarket,Sushi Restaurant,Taiwanese Restaurant,Tennis Court,Thai Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wings Joint,Yoga Studio
0,0,M1B,"Rouge, Malvern",43.806686,-79.194353,Images Salon & Spa,This spot is popular,595,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,M1B,"Rouge, Malvern",43.806686,-79.194353,Staples Morningside,This spot is popular,735,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,600,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,M1B,"Rouge, Malvern",43.806686,-79.194353,Wendy's,This spot is popular,387,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,M1B,"Rouge, Malvern",43.806686,-79.194353,Harvey's,This spot is popular,796,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [33]:
important_list_of_features = ['Neighborhood','Neighborhood Latitude','Neighborhood Longitude','African Restaurant','American Restaurant','Asian Restaurant','Bakery','Breakfast Spot',

 'Burger Joint',
 
 
 
 'Cajun / Creole Restaurant',
 'Cantonese Restaurant',
 'Caribbean Restaurant',
 'Chinese Restaurant',
 
 'Diner',


 'Fast Food Restaurant',
 'Filipino Restaurant',
 'Fish Market',
 'Food & Drink Shop',
 'Fried Chicken Joint',
 'Fruit & Vegetable Store',
 
 'Greek Restaurant',
 'Grocery Store',
 
 'Hakka Restaurant',

 'Hotpot Restaurant',
 
 'Indian Restaurant',

 'Italian Restaurant',
 'Japanese Restaurant',
 'Korean Restaurant',
 'Latin American Restaurant',



 'Malay Restaurant',
 
 'Mediterranean Restaurant',
 
 'Mexican Restaurant',
 'Middle Eastern Restaurant',
 
 'Noodle House',
 
 'Pizza Place',
 
 'Restaurant',
 'Sandwich Place',
 'Seafood Restaurant',
 'Shanghai Restaurant',
 
 'Sushi Restaurant',
 'Taiwanese Restaurant',
 
 'Thai Restaurant',
 
 'Vegetarian / Vegan Restaurant',
 
 'Vietnamese Restaurant',
 'Wings Joint']

In [34]:
scarborough_onehot = scarborough_onehot[important_list_of_features].drop(
    columns = ['Neighborhood Latitude', 'Neighborhood Longitude']).groupby(
    'Neighborhood').sum()


scarborough_onehot.head()

,African Restaurant,American Restaurant,Asian Restaurant,Bakery,Breakfast Spot,Burger Joint,Cajun / Creole Restaurant,Cantonese Restaurant,Caribbean Restaurant,Chinese Restaurant,Diner,Fast Food Restaurant,Filipino Restaurant,Fish Market,Food & Drink Shop,Fried Chicken Joint,Fruit & Vegetable Store,Greek Restaurant,Grocery Store,Hakka Restaurant,Hotpot Restaurant,Indian Restaurant,Italian Restaurant,Japanese Restaurant,Korean Restaurant,Latin American Restaurant,Malay Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Noodle House,Pizza Place,Restaurant,Sandwich Place,Seafood Restaurant,Shanghai Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wings Joint
Neighborhood,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Agincourt,0,1,0,2,1,0,0,1,2,9,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,2,1,2,1,1,1,0,0,0,0,0
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",0,0,0,2,0,0,0,0,1,4,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,2,2,0,0,0,0,0,0,0,1,0,0
"Birch Cliff, Cliffside West",0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0
Cedarbrae,0,0,1,4,0,1,0,0,1,1,0,1,0,0,0,1,0,0,1,1,0,2,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,1
"Clairlea, Golden Mile, Oakridge",0,0,0,2,0,0,0,0,0,0,1,2,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,0,0,0,0


In [35]:
feat_name_list = list(scarborough_onehot.columns)
restaurant_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Restaurant') != (-1):
        restaurant_list.append(value)
        
scarborough_onehot['Total Restaurants'] = scarborough_onehot[restaurant_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = restaurant_list)


feat_name_list = list(scarborough_onehot.columns)
joint_list = []


for counter, value in enumerate(feat_name_list):
    if value.find('Joint') != (-1):
        joint_list.append(value)
        
scarborough_onehot['Total Joints'] = scarborough_onehot[joint_list].sum(axis = 1)
scarborough_onehot = scarborough_onehot.drop(columns = joint_list)

In [36]:
scarborough_onehot

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints
Neighborhood,,,,,,,,,,,,
Agincourt,2,1,0,0,0,0,1,1,2,2,20,0
"Agincourt North, L'Amoreaux East, Milliken, Steeles East",2,0,0,0,0,0,1,2,2,0,9,0
"Birch Cliff, Cliffside West",0,0,1,0,0,0,0,0,0,0,3,0
Cedarbrae,4,0,0,0,0,0,1,0,1,0,8,3
"Clairlea, Golden Mile, Oakridge",2,0,1,0,0,0,1,0,1,1,4,0
"Clarks Corners, Sullivan, Tam O'Shanter",1,1,0,0,0,0,1,1,2,2,12,1
"Cliffcrest, Cliffside, Scarborough Village West",0,0,0,0,0,0,0,0,2,0,3,1
"Dorset Park, Scarborough Town Centre, Wexford Heights",3,0,0,0,0,0,0,0,1,1,14,4
"East Birchmount Park, Ionview, Kennedy Park",0,0,0,0,0,0,2,0,2,1,5,1


In [37]:
from sklearn.cluster import KMeans

# run k-means clustering
kmeans = KMeans(n_clusters = 5, random_state = 0).fit(scarborough_onehot)

In [38]:
means_df = pd.DataFrame(kmeans.cluster_centers_)
means_df.columns = scarborough_onehot.columns
means_df.index = ['G1','G2','G3','G4','G5']
means_df['Total Sum'] = means_df.sum(axis = 1)
means_df.sort_values(axis = 0, by = ['Total Sum'], ascending=False)

,Bakery,Breakfast Spot,Diner,Fish Market,Food & Drink Shop,Fruit & Vegetable Store,Grocery Store,Noodle House,Pizza Place,Sandwich Place,Total Restaurants,Total Joints,Total Sum
G2,2.0,1.00,0.00,0.0,0.00,0.0,1.00,1.0,2.0,2.0,20.0,0.00,29.0
G4,2.0,0.50,0.00,0.0,0.00,0.0,0.50,0.5,1.5,1.5,13.0,2.50,22.0
G1,1.6,0.40,0.00,0.2,0.00,0.2,1.00,0.4,1.2,0.4,8.2,1.00,14.6
G3,0.5,0.00,0.25,0.0,0.25,0.0,1.25,0.0,2.0,1.0,4.5,0.75,10.5
G5,0.0,0.25,0.25,0.0,0.00,0.0,0.00,0.0,0.5,0.0,2.5,0.50,4.0


In [45]:
neigh_summary = pd.DataFrame([scarborough_onehot.index, 1 + kmeans.labels_]).T
neigh_summary.columns = ['Neighborhood', 'Group']
neigh_summary

,Neighborhood,Group
0,Agincourt,2
1,"Agincourt North, L'Amoreaux East, Milliken, St...",1
2,"Birch Cliff, Cliffside West",5
3,Cedarbrae,1
4,"Clairlea, Golden Mile, Oakridge",3
5,"Clarks Corners, Sullivan, Tam O'Shanter",4
6,"Cliffcrest, Cliffside, Scarborough Village West",5
7,"Dorset Park, Scarborough Town Centre, Wexford ...",4
8,"East Birchmount Park, Ionview, Kennedy Park",3
9,"Guildwood, Morningside, West Hill",3


In [46]:
neigh_summary[neigh_summary['Group'] == 4]

,Neighborhood,Group
5,"Clarks Corners, Sullivan, Tam O'Shanter",4
7,"Dorset Park, Scarborough Town Centre, Wexford ...",4


In [47]:
neigh_summary[neigh_summary['Group'] == 5]

,Neighborhood,Group
2,"Birch Cliff, Cliffside West",5
6,"Cliffcrest, Cliffside, Scarborough Village West",5
10,"Highland Creek, Rouge Hill, Port Union",5
15,Woburn,5


In [48]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 5]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1N',
 'Neighborhood': 'Birch Cliff, Cliffside West',
 'Neighborhood Latitude': 43.692657000000004,
 'Neighborhood Longitude': -79.2648481}

In [49]:
neigh_summary[neigh_summary['Group'] == 1]

,Neighborhood,Group
1,"Agincourt North, L'Amoreaux East, Milliken, St...",1
3,Cedarbrae,1
11,"L'Amoreaux West, Steeles West",1
12,"Maryvale, Wexford",1
13,"Rouge, Malvern",1


In [50]:
neigh_summary[neigh_summary['Group'] == 4]

,Neighborhood,Group
5,"Clarks Corners, Sullivan, Tam O'Shanter",4
7,"Dorset Park, Scarborough Town Centre, Wexford ...",4


In [51]:
name_of_neigh = list(neigh_summary[neigh_summary['Group'] == 4]['Neighborhood'])[0]
scarborough_venues[scarborough_venues['Neighborhood'] == name_of_neigh].iloc[0,1:5].to_dict()

{'Postal Code': 'M1T',
 'Neighborhood': "Clarks Corners, Sullivan, Tam O'Shanter",
 'Neighborhood Latitude': 43.7816375,
 'Neighborhood Longitude': -79.3043021}